In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!cp -r "/content/gdrive/MyDrive/final_antispoofing.zip" "/content"

In [3]:
import zipfile
archive = zipfile.ZipFile('/content/final_antispoofing.zip')
archive.extractall('/content')

In [4]:
dataset_dir = '/content/final_antispoofing'
train_dataset_dir = '/content/final_antispoofing/train'
test_dataset_dir = '/content/final_antispoofing/test'

In [5]:
import os
os.mkdir('/content/antispoofing_dataset')
os.mkdir('/content/antispoofing_dataset/train')
os.mkdir('/content/antispoofing_dataset/test')
os.mkdir('/content/antispoofing_dataset/train/real')
os.mkdir('/content/antispoofing_dataset/train/spoof')
os.mkdir('/content/antispoofing_dataset/test/real')
os.mkdir('/content/antispoofing_dataset/test/spoof')

In [6]:
train_dir = '/content/antispoofing_dataset/train'
test_dir = '/content/antispoofing_dataset/test'

copying the dataset from older folder to newer folder

In [7]:
import shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [8]:
def train_test_splits(data_directory):
  for split_type in os.listdir(data_directory):
    path_to_split_type = os.path.join(data_directory,split_type)
    for category in os.listdir(path_to_split_type):
      path_to_category = os.path.join(path_to_split_type,category)
      for subject in os.listdir(path_to_category):
        path_to_subject = os.path.join(path_to_category,subject)
        for img in os.listdir(path_to_subject):
          if split_type == 'train':
            shutil.copy(os.path.join(path_to_subject,img),os.path.join(train_dir,category,img))
          else:
            shutil.copy(os.path.join(path_to_subject,img),os.path.join(test_dir,category,img))
            
  

In [9]:
train_test_splits(data_directory=dataset_dir)

In [10]:
categories = ['real','spoof']

In [11]:
print("-----------------------Exploring Training Datasets-------------------")
for category in categories:
  path=os.path.join(train_dir,category)
  if category == 'real':
    r1 = len(os.listdir(path))
  else:
    s1 = len(os.listdir(path))
  print('There are {} images in {} directory'.format(len(os.listdir(path)),category))
print('There are {} total images in training directory'.format(r1+s1))
print("-----------------------Exploring Testing Datasets-------------------")
for category in categories:
  path=os.path.join(test_dir,category)
  if category == 'real':
    r2 = len(os.listdir(path))
  else:
    s2 = len(os.listdir(path))
  print('There are {} images in {} directory'.format(len(os.listdir(path)),category))
print('There are {} total images in testing directory'.format(r2+s2))

-----------------------Exploring Training Datasets-------------------
There are 2102 images in real directory
There are 2118 images in spoof directory
There are 4220 total images in training directory
-----------------------Exploring Testing Datasets-------------------
There are 477 images in real directory
There are 474 images in spoof directory
There are 951 total images in testing directory


In [12]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [13]:
train_datagen = ImageDataGenerator(brightness_range = (0.8,1.2),
                                   rotation_range = 30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest',
                                   rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory(train_dir,target_size = (160,160),
                                                 color_mode='rgb',
                                                 class_mode='binary',batch_size=25,
                                                 shuffle=True)

Found 4220 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory(test_dir,target_size = (160,160),
                                                 color_mode='rgb',
                                                 class_mode='binary',batch_size=25)

Found 951 images belonging to 2 classes.


Building CNN

In [15]:
# initializing the CNN
cnn = tf.keras.models.Sequential()
# Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[160,160,3]))
# MaxPooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
# Adding a second Convolution Layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
# flattening
cnn.add(tf.keras.layers.Flatten())

In [16]:
# Full Connection
cnn.add(tf.keras.layers.Dense(units=120,kernel_initializer='he_normal',activation='relu'))
# Hidden layer
cnn.add(tf.keras.layers.Dense(units=120,kernel_initializer='he_normal',activation='relu'))
# Hidden layer
cnn.add(tf.keras.layers.Dense(units=120,kernel_initializer='he_normal',activation='relu'))
# Hidden layer
cnn.add(tf.keras.layers.Dense(units=120,kernel_initializer='he_normal',activation='relu'))
# Output layer
cnn.add(tf.keras.layers.Dense(units=1,kernel_initializer='glorot_normal',activation='sigmoid'))

In [17]:
cnn.compile(optimizer="adam", loss="binary_crossentropy", metrics=['accuracy'])

In [18]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 158, 158, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 79, 79, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 77, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 38, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 46208)             0         
                                                                 
 dense (Dense)               (None, 120)               5

In [19]:
import os
os.mkdir('/content/model_weights')

In [20]:
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
import json
model_checkpoint = ModelCheckpoint('./model_weights/finalyearproject_antispoofing_model_{epoch:02d}-{val_accuracy:.6f}.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True,save_weights_only=True)

In [21]:
from keras import callbacks
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 100,callbacks=[model_checkpoint])

Epoch 1/100
169/169 [==============================] - ETA: 0s - loss: 0.4668 - accuracy: 0.7692
Epoch 00001: val_loss improved from inf to 0.57745, saving model to ./model_weights/finalyearproject_antispoofing_model_01-0.797056.h5
169/169 [==============================] - 49s 212ms/step - loss: 0.4668 - accuracy: 0.7692 - val_loss: 0.5774 - val_accuracy: 0.7971
Epoch 2/100
169/169 [==============================] - ETA: 0s - loss: 0.2607 - accuracy: 0.8924
Epoch 00002: val_loss improved from 0.57745 to 0.29508, saving model to ./model_weights/finalyearproject_antispoofing_model_02-0.845426.h5
169/169 [==============================] - 36s 211ms/step - loss: 0.2607 - accuracy: 0.8924 - val_loss: 0.2951 - val_accuracy: 0.8454
Epoch 3/100
169/169 [==============================] - ETA: 0s - loss: 0.2488 - accuracy: 0.8988
Epoch 00003: val_loss improved from 0.29508 to 0.21772, saving model to ./model_weights/finalyearproject_antispoofing_model_03-0.899054.h5
169/169 [===================

In [22]:
#serialize model to json
model_json = cnn.to_json()
with open('finalyearproject_antispoofing_cnn_model.json','w') as json_file:
  json_file.write(model_json)

In [27]:
from keras.preprocessing import image
import numpy as np
def check_fakes(path,category_type):
  predictor = {}
  path = os.path.join(path,category_type)
  for img in os.listdir(path):
    try:
      img = image.load_img(os.path.join(path,img),target_size=(160,160))
      img = image.img_to_array(img)
      img = np.expand_dims(img,axis=0)
      img = img/255.0
      prediction = cnn.predict(img)
      if prediction > 0.5:
        prediction_class = 1
      else:
        prediction_class = 0
      result = categories[prediction_class]
      if result not in predictor:
        predictor[result] = 1
      else:
        predictor[result] += 1
    except Exception as e:
      pass
  return predictor
      

In [28]:
check_fakes(test_dir,categories[1]) # testing for spoof images

{'real': 7, 'spoof': 467}

In [29]:
check_fakes(test_dir,categories[0]) # testing for real images

{'real': 452, 'spoof': 25}